In [1]:
import geopandas as gpd
import pandas as pd
import os
from glob import glob
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error

In [2]:
cells = gpd.read_file('global-inputs/HR6-change-cells-aoi.gpkg')

cell_ids = cells.hex_id.tolist()

In [3]:
shoreline_files = pd.Series(sorted(glob("data/HR6/*/shoreline_timeseries_tidal_correction.csv")))

In [4]:
def calc_trends(file):
    df  = pd.read_csv(file)
    try:
        df.date = pd.to_datetime(df.date)
    except:
        print(file)
    df.index = (df.date - df.date.min()).dt.days / 365.25
    df.drop(columns=['date', 'Unnamed: 0', 'corrected_IW_shoreline_position', 'EOV_shoreline_chg'], inplace=True, errors='ignore')
    trends = {}
    for col in df.columns:
        sub_df = df[col].dropna()
        
        if not len(sub_df):
            continue
        x = sub_df.index.to_numpy().reshape(-1,1)
        y = sub_df
        linear_trend = LinearRegression().fit(x,y)
        pred = linear_trend.predict(x)

        trends['index'] = file.split('/')[2]
        if col == 'normalised_IW_shoreline_position':
            trends["IW shoreline trend"] = linear_trend.coef_[0]
            trends["IW shoreline intercept"] = linear_trend.intercept_
            trends["IW shoreline n_points"] = len(df[col])
            trends["IW shoreline n_points_nonan"] = len(sub_df)
            trends["IW shoreline r2_score"] = r2_score(y, pred)
            trends["IW shoreline mae"] = mean_absolute_error(y, pred)
            trends["IW shoreline mse"] = mean_squared_error(y, pred)
            trends["IW shoreline rmse"] = root_mean_squared_error(y, pred)
        else:
            trends["EOV shoreline trend"] = linear_trend.coef_[0]
            trends["EOV shoreline intercept"] = linear_trend.intercept_
            trends["EOV shoreline n_points"] = len(df[col])
            trends["EOV shoreline n_points_nonan"] = len(sub_df)
            trends["EOV shoreline r2_score"] = r2_score(y, pred)
            trends["EOV shoreline mae"] = mean_absolute_error(y, pred)
            trends["EOV shoreline mse"] = mean_squared_error(y, pred)
            trends["EOV shoreline rmse"] = root_mean_squared_error(y, pred)

    return pd.DataFrame([trends])
    

In [5]:
trends = pd.concat([calc_trends(f) for f in shoreline_files])

trends.dropna(inplace=True)

trends

/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


,index,IW shoreline trend,IW shoreline intercept,IW shoreline n_points,IW shoreline n_points_nonan,IW shoreline r2_score,IW shoreline mae,IW shoreline mse,IW shoreline rmse,EOV shoreline trend,EOV shoreline intercept,EOV shoreline n_points,EOV shoreline n_points_nonan,EOV shoreline r2_score,EOV shoreline mae,EOV shoreline mse,EOV shoreline rmse
0,86ba24017ffffff,0.671251,4.005472,176.0,176.0,0.047041,11.922584,189.714321,13.773682,-0.532692,0.671142,176.0,176.0,0.007992,22.609881,732.083319,27.057038
0,86ba24037ffffff,1.041432,-6.280012,166.0,166.0,0.347994,6.966765,67.855639,8.237453,1.585770,-20.751274,166.0,166.0,0.426320,5.583544,112.995230,10.629921
0,86ba24067ffffff,0.404550,15.703989,225.0,225.0,0.009197,9.960040,389.017429,19.723525,0.628829,-17.224565,225.0,225.0,0.009132,23.405065,946.705355,30.768577
0,86ba2414fffffff,0.333610,5.501252,193.0,193.0,0.016566,6.511301,100.795249,10.039684,0.600633,-15.201153,193.0,193.0,0.075426,7.163161,67.465139,8.213716
0,86ba2415fffffff,0.383988,0.994078,307.0,307.0,0.005552,14.730841,717.037423,26.777554,-0.202294,-11.981312,307.0,307.0,0.000800,26.542546,1388.440320,37.261781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,86dab3707ffffff,0.776031,23.187273,237.0,237.0,0.066894,18.979537,489.695383,22.129062,0.179149,13.659885,237.0,237.0,0.073041,3.444084,23.743796,4.872761
0,86dab370fffffff,0.269206,27.797196,213.0,213.0,0.013518,15.707413,317.843751,17.828173,-0.172802,19.689800,213.0,213.0,0.004657,13.725087,383.570035,19.584944
0,86dab3737ffffff,1.926790,-13.576790,172.0,172.0,0.636867,7.813505,121.219279,11.009963,0.061498,-1.089441,172.0,172.0,0.038141,1.001072,5.461649,2.337017
0,86dab3757ffffff,-0.744676,23.184835,207.0,207.0,0.090247,15.028589,323.455289,17.984863,0.324783,9.978654,207.0,207.0,0.068491,7.616960,83.009131,9.110935


In [6]:
trends_file_path = 'shoreline_trends_by_cell.geojson'
# if os.path.exists(trends_file_path):
#     cell_trends = gpd.read_file('shoreline_trends_by_cell.geojson')
#     cell_trends.to_crs("EPSG:4326", inplace=True)
#     cell_trends.update(trends)
#     cell_trends.to_file(trends_file_path)
# else:
cell_trends = cells.merge(trends, on='index')
cell_trends.to_crs("EPSG:4326", inplace=True)
cell_trends.to_file(trends_file_path)
    
# cell_trends['hex_id'].to_csv("global-inputs/valid_cells.csv")

cell_trends

,index,hex_id,parent_id,sediment_area_ha,geometry,IW shoreline trend,IW shoreline intercept,IW shoreline n_points,IW shoreline n_points_nonan,IW shoreline r2_score,...,IW shoreline mse,IW shoreline rmse,EOV shoreline trend,EOV shoreline intercept,EOV shoreline n_points,EOV shoreline n_points_nonan,EOV shoreline r2_score,EOV shoreline mae,EOV shoreline mse,EOV shoreline rmse
0,86bb5e18fffffff,86bb5e18fffffff,85bb5e1bfffffff,16.76,"POLYGON ((173.94469 -35.12657, 173.94025 -35.0...",0.270699,-4.102205,136.0,136.0,0.024654,...,104.796870,10.237034,1.104938,-20.470234,136.0,136.0,0.048329,22.475745,869.082522,29.480206
1,86bb5e19fffffff,86bb5e19fffffff,85bb5e1bfffffff,20.00,"POLYGON ((174.01618 -35.12027, 174.01171 -35.0...",0.496066,2.480027,84.0,84.0,0.084891,...,56.594961,7.522962,0.431622,-3.907391,84.0,84.0,0.034109,9.840199,112.552528,10.609078
2,86bb58027ffffff,86bb58027ffffff,85bb5803fffffff,97.20,"POLYGON ((172.71095 -34.47891, 172.70704 -34.4...",0.826371,-0.681890,7.0,7.0,0.081622,...,29.049672,5.389775,3.016152,8.967772,7.0,7.0,0.070048,18.404964,456.612787,21.368500
3,86bb72b4fffffff,86bb72b4fffffff,85bb72b7fffffff,14.08,"POLYGON ((173.87492 -39.18614, 173.87027 -39.1...",-0.087499,22.536046,155.0,155.0,0.000283,...,656.276965,25.617903,1.790282,-58.423393,155.0,155.0,0.047987,34.331471,1540.400634,39.247938
4,86bb2bae7ffffff,86bb2bae7ffffff,85bb2baffffffff,44.20,"POLYGON ((175.14401 -40.80153, 175.13871 -40.7...",1.423731,2.400355,98.0,98.0,0.185166,...,466.764080,21.604724,0.859078,-8.024577,98.0,98.0,0.134504,11.058167,248.499359,15.763862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,86da9285fffffff,86da9285fffffff,85da9287fffffff,2.00,"POLYGON ((171.98283 -41.59482, 171.97888 -41.5...",-0.317627,6.649636,256.0,256.0,0.014664,...,425.061420,20.617018,0.081296,10.790518,256.0,256.0,0.071387,1.309099,5.390592,2.321765
631,86da94ab7ffffff,86da94ab7ffffff,85da94abfffffff,50.36,"POLYGON ((172.82222 -43.2102, 172.81779 -43.17...",-0.194254,-10.164818,175.0,175.0,0.007225,...,345.565446,18.589391,0.330989,-2.666553,175.0,175.0,0.203491,4.341013,28.577840,5.345825
632,86da91d8fffffff,86da91d8fffffff,85da91dbfffffff,5.36,"POLYGON ((171.05489 -42.73298, 171.05131 -42.6...",-0.775836,11.784112,241.0,241.0,0.056748,...,438.801779,20.947596,0.092224,-0.338082,241.0,241.0,0.291152,0.801414,0.908177,0.952983
633,86bb299afffffff,86bb299afffffff,85bb299bfffffff,11.68,"POLYGON ((174.2819 -41.84778, 174.27691 -41.81...",2.837806,-40.418872,328.0,328.0,0.293555,...,668.016689,25.846019,-4.488396,21.394402,328.0,328.0,0.183639,42.939511,3086.969259,55.560501
